# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 29 2022 11:56AM,254004,10,SO94484,"Senseonics, Incorporated",Released
1,Dec 29 2022 11:56AM,254004,10,SO94485,"Senseonics, Incorporated",Released
2,Dec 29 2022 11:56AM,254004,10,SO94486,"Senseonics, Incorporated",Released
3,Dec 29 2022 11:56AM,254004,10,SO94487,"Senseonics, Incorporated",Released
4,Dec 29 2022 11:02AM,254002,10,PRF-45581,Bio-PRF,Released
5,Dec 29 2022 11:02AM,254002,10,PRF-45589,Bio-PRF,Released
6,Dec 29 2022 11:02AM,254002,10,PRF-45592,Bio-PRF,Released
7,Dec 29 2022 11:02AM,254002,10,PRF-45596,Bio-PRF,Released
8,Dec 29 2022 11:02AM,254002,10,PRF-45599,Bio-PRF,Released
9,Dec 29 2022 11:02AM,254002,10,PRF-45603,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
26,253999,Released,1
27,254000,Released,1
28,254001,Released,1
29,254002,Released,16
30,254004,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253999,NaN,1.0
254000,NaN,1.0
254001,NaN,1.0
254002,NaN,16.0
254004,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253926,0.0,5.0
253929,0.0,1.0
253930,0.0,1.0
253939,0.0,23.0
253943,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253926,0,5
253929,0,1
253930,0,1
253939,0,23
253943,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253926,0,5
1,253929,0,1
2,253930,0,1
3,253939,0,23
4,253943,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253926,,5
1,253929,,1
2,253930,,1
3,253939,,23
4,253943,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 29 2022 11:56AM,254004,10,"Senseonics, Incorporated"
4,Dec 29 2022 11:02AM,254002,10,Bio-PRF
20,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc."
21,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.
33,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc."
34,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.
35,Dec 29 2022 10:34AM,253995,10,"Methapharm, Inc."
55,Dec 29 2022 10:31AM,253996,19,"NAPP Technologies, LLC"
56,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc."
62,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 29 2022 11:56AM,254004,10,"Senseonics, Incorporated",,4
1,Dec 29 2022 11:02AM,254002,10,Bio-PRF,,16
2,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",,1
3,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,,12
4,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",,1
5,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,,1
6,Dec 29 2022 10:34AM,253995,10,"Methapharm, Inc.",,20
7,Dec 29 2022 10:31AM,253996,19,"NAPP Technologies, LLC",,1
8,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",,6
9,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 11:56AM,254004,10,"Senseonics, Incorporated",4,
1,Dec 29 2022 11:02AM,254002,10,Bio-PRF,16,
2,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1,
3,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12,
4,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1,
5,Dec 29 2022 10:38AM,253999,10,Eywa Pharma Inc.,1,
6,Dec 29 2022 10:34AM,253995,10,"Methapharm, Inc.",20,
7,Dec 29 2022 10:31AM,253996,19,"NAPP Technologies, LLC",1,
8,Dec 29 2022 10:19AM,253994,10,"Methapharm, Inc.",6,
9,Dec 29 2022 10:11AM,253992,17,"Breckenridge Pharmaceutical, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 11:56AM,254004,10,"Senseonics, Incorporated",4,
1,Dec 29 2022 11:02AM,254002,10,Bio-PRF,16,
2,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1,
3,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12,
4,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 11:56AM,254004,10,"Senseonics, Incorporated",4.0,NaN
1,Dec 29 2022 11:02AM,254002,10,Bio-PRF,16.0,NaN
2,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1.0,NaN
3,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12.0,NaN
4,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 11:56AM,254004,10,"Senseonics, Incorporated",4.0,0.0
1,Dec 29 2022 11:02AM,254002,10,Bio-PRF,16.0,0.0
2,Dec 29 2022 10:41AM,254001,21,"NBTY Global, Inc.",1.0,0.0
3,Dec 29 2022 10:41AM,253997,10,Eywa Pharma Inc.,12.0,0.0
4,Dec 29 2022 10:40AM,254000,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5841233,149.0,13.0
17,507950,2.0,0.0
19,761961,3.0,0.0
20,253974,3.0,0.0
21,508001,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5841233,149.0,13.0
1,17,507950,2.0,0.0
2,19,761961,3.0,0.0
3,20,253974,3.0,0.0
4,21,508001,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,149.0,13.0
1,17,2.0,0.0
2,19,3.0,0.0
3,20,3.0,0.0
4,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,149.0
1,17,Released,2.0
2,19,Released,3.0
3,20,Released,3.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,17,19,20,21
Status,,,,,
Executing,13.0,0.0,0.0,0.0,0.0
Released,149.0,2.0,3.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,17,19,20,21
0,Executing,13.0,0.0,0.0,0.0,0.0
1,Released,149.0,2.0,3.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,17,19,20,21
0,Executing,13.0,0.0,0.0,0.0,0.0
1,Released,149.0,2.0,3.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()